# Creating Raster Information Product using Raster GeoAnalytics

This notebook shows how image services can be used.

In [1]:
from arcgis.gis import GIS
from arcgis.lyr import ImageLayer
from IPython.display import display

gis = GIS()

Here we're searchcing for multispectral landsat imagery layer:

In [2]:
items = gis.content.search("Landsat 8 Views", max_items=2)

In [3]:
for item in items:
    display(item)

<Item title:"Imagery" type:Layer owner:esri>

<Item title:"Landsat 8 Views" type:Image Service owner:esri>

In [4]:
landsat = items[1]

In [5]:
imglyr = ImageLayer(landsat)

This layer has been published with several Raster Functions, that the code below is cycling through, and listing out:

In [6]:
for fn in imglyr['rasterFunctionInfos']:
    print(fn['name'])

Agriculture with DRA
Bathymetric with DRA
Color Infrared with DRA
Natural Color with DRA
Short-wave Infrared with DRA
Geology with DRA
Agriculture
Bathymetric
Color Infrared
Geology
Natural Color
Short-wave Infrared
NDVI Colorized
Normalized Difference Moisture Index Colorized
NDVI Raw
NBR Raw
None


In [7]:
map = gis.map("Pallikaranai", 13)

:0: FutureWarning: IPython widgets are experimental and may change in the future.


In [8]:
map

In [9]:
map.add_layer(imglyr)

The utility of raster functions is better seen when we interactively cycle through these raster functions and apply them to the map, like the code below does. This is using on-the-fly image processing at display resolution to cycle through the various raster functions, and showing how the layer can be visualized using these different raster functions published with the layer.

In [10]:
import time

for fn in imglyr['rasterFunctionInfos'][:6]:
    print(fn['name'])
    map.remove_layers()
    map.add_layer(imglyr, {"imageServiceParameters" :{ "renderingRule": { "rasterFunction": fn['name']}}})
    time.sleep(2)
    

Agriculture with DRA
Bathymetric with DRA
Color Infrared with DRA
Natural Color with DRA
Short-wave Infrared with DRA
Geology with DRA


In [11]:
map = gis.map("Pallikaranai", 13)
map

Developers can create their own raster functions, by chaining different raster functions. For instance, the code below is doing an Extract Band and extracting out the [4,5,3] band combination, and applying a Stretch to get the land-water boundary visualization that makes it easy to see where land is and where water is:

In [12]:
raster_fn = {
               "rasterFunction": "Stretch",
               "rasterFunctionArguments":{
                    "Raster":{
                        "rasterFunction": "ExtractBand",
                        "rasterFunctionArguments":{"BandIds": [4,5,3]}
                    },
                    "StretchType": 6,
                    "DRA": True,
                    "Gamma": [1,1,1],
                    "UseGamma": True
                },
               "outputPixelType":"U8"
              }

In [13]:
map.add_layer(landsat, {"imageServiceParameters" :{ "renderingRule": raster_fn }})

# Creating a Raster Information Product using Landsat 8 imagery

This part of the notebook shows how Raster GeoAnalytics (in ArcGIS Portal 10.5 Beta) can be used to generate a raster information product, by applying the same raster function across the extent of an image service on my Portal, at source resolution, and this can create an Information Product, that can be used for further analysis and visualization.

In [26]:
portal = GIS("https://dev06999.esri.com/portal", "admin", "changeit")

In [27]:
chennai_ls = portal.content.search("ChennaiMultispectralLandsat")[0]

In [28]:
map = portal.map("Pallikaranai, Tamil Nadu, India", 13)
map

In [29]:
map.add_layer(chennai_ls)

https://Dev06999.esri.com/server/rest/services/ChennaiMultispectralLandsat/ImageServer


In [30]:
product = portal.rasters.generate(raster_fn, "LandsatWaterBoundary3", chennai_ls)

Submitted.
Executing...
Executing (GenerateRaster): GenerateRaster {"rasterFunctionArguments":{"Raster":{"rasterFunctionArguments":{"BandIds":[4,5,3]},"rasterFunction":"ExtractBand"},"Gamma":[1,1,1],"StretchType":6,"DRA":true,"UseGamma":true},"outputPixelType":"U8","rasterFunction":"Stretch"} {"itemId":"3017b6bf0a824185928ebd80c53617d8"} {"Raster":{"itemId":"2156e1ce07ea40da8a24d36271e03c3d"}} # # #
Start Time: Tue Dec 15 06:10:55 2015
Running script GenerateRaster...
Output item id is: 3017b6bf0a824185928ebd80c53617d8
Output image service url is: https://Dev06999.esri.com:6443/arcgis/rest/services/Hosted/LandsatWaterBoundary3/ImageServer
Output image service admin url is: https://Dev06999.esri.com:6443/arcgis/admin/services/Hosted/LandsatWaterBoundary3.ImageServer
Output cloud raster name is: id_3017b6bf0a824185928ebd80c53617d8
Raster https://Dev06999.esri.com:6443/arcgis/rest/services/ChennaiMultispectralLandsat/ImageServer

Using input raster dataset's spatial reference.
Data store 

In [31]:
product

{}

In [32]:
map.add_layer(product)

http://Dev06999.esri.com/server/rest/services/Hosted/LandsatWaterBoundary3/ImageServer
